In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [2]:
# Set directories
frames_dir = 'sign_language_data/frames'
labels_file = 'sign_language_data/labels.csv'

# Load and preprocess data
data = []
labels = []

for frame_file in os.listdir(frames_dir):
    gloss = frame_file.split('_')[0]
    frame = cv2.imread(os.path.join(frames_dir, frame_file))
    frame = cv2.resize(frame, (64, 64))
    data.append(frame)
    labels.append(gloss)

data = np.array(data, dtype='float32') / 255.0
labels = pd.Series(labels)
labels_encoded = pd.factorize(labels)[0]
labels_categorical = to_categorical(labels_encoded)

In [4]:
import json
# Save label encoding for later use
label_mapping = dict(zip(labels.factorize()[1], range(len(labels.factorize()[1]))))
with open('sign_language_data/label_mapping.json', 'w') as f:
    json.dump(label_mapping, f)

In [5]:
# Split data into training and validation sets
split_ratio = 0.8
split_index = int(len(data) * split_ratio)
x_train, x_val = data[:split_index], data[split_index:]
y_train, y_val = labels_categorical[:split_index], labels_categorical[split_index:]

In [6]:
# Data augmentation
train_datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1, horizontal_flip=True)
val_datagen = ImageDataGenerator()

In [7]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(label_mapping), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Train the model
batch_size = 32
model.fit(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    validation_data=val_datagen.flow(x_val, y_val),
    epochs=10
)

Epoch 1/10


c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - accuracy: 0.0315 - loss: 4.1146 - val_accuracy: 0.0000e+00 - val_loss: 4.8095
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.0642 - loss: 3.8463 - val_accuracy: 0.0000e+00 - val_loss: 6.4663
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 0.1275 - loss: 3.4458 - val_accuracy: 0.0000e+00 - val_loss: 8.8989
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.1427 - loss: 3.0349 - val_accuracy: 0.0000e+00 - val_loss: 12.5842
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.2723 - loss: 2.5373 - val_accuracy: 0.0000e+00 - val_loss: 14.5905
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.3989 - loss: 2.0690 - val_accuracy: 0.0000e+00 - val_loss: 19.0730
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.4609 - loss: 1.8743 - val_accuracy: 0.0000e+00 - val_loss: 15.3232
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.4905 - loss: 1.6239 - va

In [9]:
model.save('sign_language_model.h5')